In [71]:
import networkx as nx # Importa la librería networkx para trabajar con grafos (redes)
# import matplotlib.pyplot as plt # Importa matplotlib.pyplot para visualización (algunas partes pueden estar comentadas o reemplazadas por plotly)
# import matplotlib.patches as mpatches # Importa parches de matplotlib para elementos gráficos (posiblemente para leyendas antiguas)
import string # Importa la librería string para manejar cadenas de texto y alfabetos
import random # Importa la librería random para generar números aleatorios y selecciones al azar
import hashlib # Importa la librería hashlib para funciones de hashing seguras como SHA256
from Crypto.PublicKey import RSA # Importa la clave pública de RSA de la librería PyCryptodome para cifrado asimétrico
from Crypto.Cipher import PKCS1_OAEP # Importa el cifrado PKCS1_OAEP de PyCryptodome para cifrado RSA seguro
import binascii # Importa la librería binascii para conversiones entre binario y ASCII binario
# import matplotlib.animation as animation # Importa la librería de animación de matplotlib (posiblemente no utilizada activamente ahora)
import plotly.graph_objects as go # Importa graph_objects de plotly para crear gráficos interactivos

# Definimos el alfabeto que se utilizará para el cifrado Vigenère
ALFABETO = string.ascii_letters + string.digits + string.punctuation + ' '

# Coordenadas geográficas de las ciudades colombianas para la visualización
CIUDADES_COORDS = {
    "Bogotá": (4.711, -74.0721),
    "Medellín": (6.2442, -75.5812),
    "Cali": (3.4516, -76.5319),
    "Barranquilla": (10.9685, -74.7813),
    "Cartagena": (10.3910, -75.4794),
    "Bucaramanga": (7.1193, -73.1227),
    "Cúcuta": (7.8939, -72.5078),
    "Pereira": (4.8087, -75.6906),
    "Manizales": (5.0689, -75.5174),
    "Ibagué": (4.4389, -75.2322),
    "Pasto": (1.2136, -77.2811),
    "Neiva": (2.9965, -75.2908),
    "Villavicencio": (4.1438, -73.6224),
    "Santa Marta": (11.2408, -74.2048),
    "Montería": (8.7609, -75.8822),
    "Armenia": (4.5388, -75.6816),
    "Sincelejo": (9.3047, -75.3973),
    "Popayán": (2.4410, -76.6029),
    "Tunja": (5.5350, -73.3677),
    "Riohacha": (11.5444, -72.9078),
    "Valledupar": (10.4634, -73.2934),
    "Apartadó": (7.8834, -76.6117),
    "Florencia": (1.6137, -75.6060),
    "Quibdó": (5.6932, -76.6585),
    "Arauca": (7.0841, -70.7611),
    "San Andrés": (12.5833, -81.7000),
    "Leticia": (-4.2167, -69.9333)
}

def vigenere_cifrar(mensaje, clave):
    """
    Cifra un mensaje usando el cifrado Vigenère con una clave dada.

    Toma un mensaje y una clave como entrada. Itera sobre cada carácter del mensaje y,
    si el carácter se encuentra en el ALFABETO definido, lo cifra utilizando la clave
    y la posición del carácter en el alfabeto. Los caracteres que no están en el
    alfabeto se dejan sin cifrar.

    Args:
        mensaje (str): El mensaje a cifrar.
        clave (str): La clave para el cifrado Vigenère.

    Returns:
        str: El mensaje cifrado.
    """
    # Itera sobre cada carácter del mensaje con su índice
    return ''.join(
        # Cifra el carácter si está en el alfabeto
        ALFABETO[(ALFABETO.index(c) + ALFABETO.index(clave[i % len(clave)])) % len(ALFABETO)]
        if c in ALFABETO else c # Deja el carácter sin cifrar si no está en el alfabeto
        for i, c in enumerate(mensaje)
    )

def vigenere_descifrar(mensaje, clave):
    """
    Descifra un mensaje usando el cifrado Vigenère con una clave dada.

    Toma el mensaje cifrado y la misma clave utilizada para cifrarlo. Itera sobre
    cada carácter del mensaje cifrado y, si está en el ALFABETO, lo descifra
    utilizando la clave de forma inversa al cifrado. Los caracteres no presentes
    en el alfabeto se mantienen igual.

    Args:
        mensaje (str): El mensaje a descifrar.
        clave (str): La clave utilizada para el cifrado Vigenère.

    Returns:
        str: El mensaje descifrado.
    """
    # Itera sobre cada carácter del mensaje cifrado con su índice
    return ''.join(
        # Descifra el carácter si está en el alfabeto
        ALFABETO[(ALFABETO.index(c) - ALFABETO.index(clave[i % len(clave)])) % len(ALFABETO)]
        if c in ALFABETO else c # Deja el carácter sin descifrar si no está en el alfabeto
        for i, c in enumerate(mensaje)
    )

def firmar_mensaje(mensaje):
    """
    Firma digitalmente un mensaje calculando su hash SHA256 y adjuntando los primeros 8 caracteres.

    Simula una firma digital simple. Toma un mensaje, calcula su hash SHA256 y luego
    toma los primeros 8 caracteres de ese hash. Devuelve el mensaje original
    concatenado con "||" y estos 8 caracteres del hash como la "firma". Esto permite
    verificar si el mensaje fue alterado posteriormente (aunque no proporciona
    autenticación real como una firma digital con claves privadas/públicas).

    Args:
        mensaje (str): El mensaje a firmar.
    Returns:
        str: El mensaje firmado en formato "mensaje||firma".
    """
    # Calcula el hash SHA256 del mensaje
    hash_completo = hashlib.sha256(mensaje.encode()).hexdigest()
    # Toma los primeros 8 caracteres del hash
    hash8 = hash_completo[:8]
    # Concatena el mensaje original con "||" y los 8 caracteres del hash
    return f"{mensaje}||{hash8}"

def verificar_firma(mensaje_firmado):
    """
    Verifica la firma digital de un mensaje firmado.

    Verifica la "firma" de un mensaje que fue firmado previamente con firmar_mensaje.
    Toma el mensaje_firmado en formato "mensaje||firma", lo divide en el mensaje
    original y la firma. Recalcula el hash SHA256 del mensaje original extraído y
    compara los primeros 8 caracteres con la firma proporcionada.

    Args:
        mensaje_firmado (str): El mensaje firmado en formato "mensaje||firma".

    Returns:
        tuple: Una tupla que contiene un booleano (True si la firma es válida, False si no)
               y el mensaje original extraído.
    """
    try:
        # Divide el mensaje firmado en mensaje original y firma
        mensaje, firma = mensaje_firmado.rsplit('||', 1)
        # Recalcula el hash del mensaje original extraído y compara los primeros 8 caracteres con la firma
        return firma == hashlib.sha256(mensaje.encode()).hexdigest()[:8], mensaje
    except ValueError:
        # Si no se puede dividir el mensaje (formato incorrecto), la firma es inválida
        return False, mensaje_firmado

def crear_red_dispositivos(n):
    """
    Crea una red aleatoria de dispositivos (nodos) distribuidos en ciudades colombianas.

    A cada nodo se le asigna un par de claves RSA (pública y privada) y una ubicación
    geográfica. Crea un grafo aleatorio usando el modelo de Erdos-Renyi y asegura
    que sea conexo (que todos los nodos estén conectados).

    Args:
        n (int): El número de nodos (dispositivos) a crear.

    Returns:
        nx.Graph: El grafo que representa la red de dispositivos.
    """
    # Obtiene la lista de ciudades y la mezcla aleatoriamente
    ciudades = list(CIUDADES_COORDS.keys())
    random.shuffle(ciudades)

    # Crea un grafo aleatorio usando el modelo de Erdos-Renyi
    G = nx.erdos_renyi_graph(n, 0.4)
    # Asegura que el grafo sea conexo (todos los nodos conectados)
    while not nx.is_connected(G):
        G = nx.erdos_renyi_graph(n, 0.4)

    # Asigna claves RSA y ubicación a cada nodo
    for nodo in G.nodes:
        # Genera un par de claves RSA (pública y privada)
        key = RSA.generate(2048)
        G.nodes[nodo]['rsa_private'] = key
        G.nodes[nodo]['rsa_public'] = key.publickey()
        # Asigna una ciudad al nodo
        ciudad = ciudades[nodo % len(ciudades)]
        G.nodes[nodo]['ciudad'] = ciudad
        G.nodes[nodo]['pos'] = CIUDADES_COORDS[ciudad] # Asigna coordenadas de la ciudad
    return G

def enviar_mensaje_unico(G, origen, destino, mensaje, espia_activo=True):
    """
    Simula el envío de un mensaje cifrado y firmado entre dos nodos de la red.

    Utiliza cifrado Vigenère para el mensaje y cifrado RSA para la clave simétrica.
    Opcionalmente, simula un ataque Man-in-the-Middle (MITM) donde un nodo espía
    intercepta el mensaje.

    Args:
        G (nx.Graph): El grafo que representa la red de dispositivos.
        origen (int): El índice del nodo de origen.
        destino (int): El índice del nodo de destino.
        mensaje (str): El mensaje a enviar.
        espia_activo (bool, optional): True para activar la simulación del espía, False para desactivarla.
                                       Por defecto es True.

    Returns:
        tuple: Una tupla que contiene el camino recorrido por el mensaje y el nodo espía (si está activo).
    """
    # Calcula el camino más corto entre origen y destino
    camino = nx.shortest_path(G, origen, destino)
    # Selecciona un nodo espía aleatorio en el camino (excluyendo origen y destino) si el espía está activo y el camino tiene al menos 3 nodos
    espia = random.choice(camino[1:-1]) if espia_activo and len(camino) > 2 else None

    # Genera una clave simétrica aleatoria para el cifrado Vigenère
    clave_simetrica = ''.join(random.choice(ALFABETO) for _ in range(16))
    # Firma digitalmente el mensaje usando SHA256
    mensaje_firmado = firmar_mensaje(mensaje)
    # Cifra el mensaje firmado usando Vigenère con la clave simétrica
    mensaje_cifrado = vigenere_cifrar(mensaje_firmado, clave_simetrica)

    # Obtiene la clave pública del nodo destino
    public_key = G.nodes[destino]['rsa_public']
    # Crea un objeto de cifrado RSA con PKCS1_OAEP
    cipher_rsa = PKCS1_OAEP.new(public_key)
    # Cifra la clave simétrica usando la clave pública del destino
    clave_cifrada = cipher_rsa.encrypt(clave_simetrica.encode())

    # Imprime información sobre el envío del mensaje
    print(f"\n📤 Enviando mensaje de nodo {origen} a nodo {destino}:")
    print(" - Camino:", [f"{n} ({G.nodes[n]['ciudad']})" for n in camino])
    print(" - Clave cifrada (hex):", binascii.hexlify(clave_cifrada).decode())
    print(" - Mensaje cifrado:", mensaje_cifrado)

    # Simula la intercepción por el espía si está activo
    if espia is not None:
        print(f"\n⚠️ Nodo espía interceptó el mensaje en nodo {espia} ({G.nodes[espia]['ciudad']}):")
        print(" - Contenido interceptado:", mensaje_cifrado)
        # Simula que el espía intenta descifrar la clave simétrica (para demostración, asume que el espía tiene la clave privada del destino)
        try:
            spy_private_key = G.nodes[destino]['rsa_private'] # Espía obtiene la clave privada del destino (simulación)
            spy_clave_descifrada = PKCS1_OAEP.new(spy_private_key).decrypt(clave_cifrada).decode()
            print(" - Clave simétrica descifrada (por el espía):", spy_clave_descifrada)
        except Exception as e:
            print(f" - El espía no pudo descifrar la clave simétrica. Error: {e}")

    # Obtiene la clave privada del nodo destino
    private_key = G.nodes[destino]['rsa_private']
    # Descifra la clave simétrica usando la clave privada del destino
    clave_descifrada = PKCS1_OAEP.new(private_key).decrypt(clave_cifrada).decode()
    # Descifra el mensaje cifrado usando Vigenère con la clave simétrica descifrada
    mensaje_descifrado = vigenere_descifrar(mensaje_cifrado, clave_descifrada)
    # Verifica la firma digital del mensaje descifrado
    valido, mensaje_final = verificar_firma(mensaje_descifrado)

    # Imprime información sobre el mensaje recibido
    print("\n📥 Mensaje recibido:")
    print(" - Descifrado:", mensaje_final)
    print(" - Integridad:", "✅ Válido" if valido else "❌ Alterado")

    # Devuelve el camino recorrido y el nodo espía (si existe)
    return camino, espia


# Nueva función para visualizar la red en un mapa interactivo usando Plotly
def mostrar_red_en_mapa_plotly(G, camino, origen, destino, espia=None):
    """
    Visualiza la red de dispositivos en un mapa interactivo usando Plotly.

    Args:
        G (nx.Graph): El grafo que representa la red de dispositivos.
        camino (list): La lista de nodos que forman el camino del mensaje.
        origen (int): El nodo de origen del mensaje.
        destino (int): El nodo de destino del mensaje.
        espia (int, optional): El nodo espía, si existe.
    """
    # Obtiene las posiciones de los nodos
    pos = nx.get_node_attributes(G, 'pos')
    # Crea etiquetas para los nodos con su número y ciudad
    labels = {n: f"{n} ({G.nodes[n]['ciudad']})" for n in G.nodes}

    # Prepara los datos para las aristas del grafo
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    # Crea el trazo para las aristas
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    # Prepara los datos para los nodos del grafo
    node_x = []
    node_y = []
    node_text = []
    node_color = []
    node_size = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(labels[node])
        # Asigna colores y tamaños especiales a los nodos de origen, destino y espía
        if node == origen:
            node_color.append('orange')
            node_size.append(20)
        elif node == destino:
            node_color.append('red')
            node_size.append(20)
        elif node == espia:
            node_color.append('purple')
            node_size.append(20)
        else:
            node_color.append('lightblue')
            node_size.append(10)

    # Crea el trazo para los nodos
    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text',
        text=node_text,
        textposition="bottom center",
        hoverinfo='text',
        marker=dict(
            showscale=False,
            colorscale='YlGnBu',
            reversescale=True,
            color=node_color,
            size=node_size,
            colorbar=dict(
                thickness=15,
                title='Conexiones de Nodos',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    # Crear aristas para el camino recorrido por el mensaje
    path_edge_x = []
    path_edge_y = []
    for i in range(len(camino) - 1):
        x0, y0 = pos[camino[i]]
        x1, y1 = pos[camino[i+1]]
        path_edge_x.append(x0)
        path_edge_x.append(x1)
        path_edge_x.append(None)
        path_edge_y.append(y0)
        path_edge_y.append(y1)
        path_edge_y.append(None)

    # Crea el trazo para el camino
    path_trace = go.Scatter(
        x=path_edge_x, y=path_edge_y,
        line=dict(width=2.5, color='green'),
        hoverinfo='none',
        mode='lines',
        name='Camino')

    # Crear punto animado para el mensaje que se mueve a lo largo del camino
    message_point = go.Scatter(
        x=[pos[camino[0]][0]], y=[pos[camino[0]][1]],
        mode='markers',
        marker=dict(size=10, color='darkgreen'),
        name='Mensaje')

    # Crea la figura de Plotly con los trazos
    fig = go.Figure(data=[edge_trace, node_trace, path_trace, message_point],
                 layout=go.Layout(
                    title='Red de Dispositivos y Camino del Mensaje',
                    titlefont_size=16,
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ # Removing the annotation line here
                     ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                   )

    # Crear frames para la animación
    frames = []
    frames_per_edge = 10 # Esto debe coincidir con la velocidad de animación deseada
    for i in range(len(camino) * frames_per_edge):
        edge_index = i // frames_per_edge
        if edge_index < len(camino) - 1:
            start_node = camino[edge_index]
            end_node = camino[edge_index + 1]
            start_pos = pos[start_node]
            end_pos = pos[end_node]
            t = (i % frames_per_edge) / frames_per_edge
            x = start_pos[0] + (end_pos[0] - start_pos[0]) * t
            y = start_pos[1] + (end_pos[1] - start_pos[1]) * t
            frames.append(go.Frame(data=[go.Scatter(x=[x], y=[y])]))
        else:
             # Permanecer en el último nodo para los frames restantes
            last_node = camino[-1]
            x, y = pos[last_node]
            frames.append(go.Frame(data=[go.Scatter(x=[x], y=[y])]))

    # Asigna los frames a la figura
    fig.frames = frames

    # Agregar botón de reproducción y slider para controlar la animación
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                buttons=[dict(label="Reproducir",
                              method="animate",
                              args=[None, {"frame": {"duration": 100, "redraw": True},
                                          "fromcurrent": True, "transition": {"duration": 0}}])],
                pad={"r": 10, "t": 87},
                showactive=False,
                x=0.1,
                xanchor="right",
                y=0,
                yanchor="top"
            )
        ],
        sliders=[dict(steps=[dict(args=[[f.name], {"frame": {"duration": 100, "redraw": True},
                                                   "mode": "immediate",
                                                   "transition": {"duration": 0}}],
                                  label=k,
                                  method="animate") for k, f in enumerate(fig.frames)],
                      transition=dict(duration=0),
                      x=0.1,
                      xanchor="left",
                      y=0,
                      yanchor="top")]
    )

    # Muestra la figura interactiva
    fig.show()


def configurar_y_enviar_mensaje_unico(G):
    """
    Configura y envía un solo mensaje, solicitando origen, destino y mensaje al usuario.
    """
    # Muestra los nodos disponibles y sus ciudades
    print(f"\nNodos disponibles: {[(n, G.nodes[n]['ciudad']) for n in G.nodes]}")

    # --- Entrada para el Mensaje ---
    print("\n--- Configuración del Mensaje ---")
    while True:
        try:
            # Solicita al usuario el nodo origen
            origen = int(input("Nodo origen: "))
            # Verifica si el nodo origen es válido
            if origen not in G.nodes:
                print("Nodo de origen no válido. Por favor, ingresa un nodo de la lista disponible.")
                continue
            # Solicita al usuario el nodo destino
            destino = int(input("Nodo destino: "))
            # Verifica si el nodo destino es válido
            if destino not in G.nodes:
                print("Nodo de destino no válido. Por favor, ingresa un nodo de la lista disponible.")
                continue
            # Verifica si el nodo origen y destino son diferentes
            if origen == destino:
                 print("El nodo origen y destino no pueden ser el mismo.")
                 continue
            # Sale del bucle si los nodos origen y destino son válidos y diferentes
            break
        except ValueError:
            # Maneja el error si la entrada no es un número entero
            print("Entrada inválida. Por favor, ingresa un número entero.")
        except nx.NetworkXNoPath:
            # Maneja el error si no existe un camino entre los nodos seleccionados
            print(f"No existe un camino entre el nodo {origen} y el nodo {destino}.")
        except Exception as e:
            # Maneja cualquier otro error
            print(f"Ocurrió un error: {e}")
    # Solicita al usuario el mensaje a enviar
    mensaje = input("Mensaje a enviar: ")

    # Solicita al usuario si desea activar el espía
    while True:
        activar_espia_input = input("¿Activar espía (MITM)? (s/n): ").lower()
        # Verifica si la entrada es válida ('s' o 'n')
        if activar_espia_input in ['s', 'n']:
            activar_espia = activar_espia_input == 's' # Asigna True si la entrada es 's', False si es 'n'
            break # Sale del bucle después de una entrada válida
        else:
            print("Entrada inválida. Por favor, ingresa 's' o 'n'.")

    # Llama a la función para enviar el mensaje
    camino, espia = enviar_mensaje_unico(G, origen, destino, mensaje, activar_espia)

    # Usar la nueva función de visualización de Plotly
    mostrar_red_en_mapa_plotly(G, camino, origen, destino, espia)


def main():
    """
    Función principal para ejecutar el simulador de red cifrada con menú interactivo.
    """
    print("\n=== Simulador de Red Cifrada con RSA + Vigenère ===")
    # Obtiene el número de ciudades disponibles
    num_ciudades = len(CIUDADES_COORDS)

    G = None # Inicializar G fuera del bucle del menú (el grafo se crea una vez)

    while True:
        # Solo crear el grafo si aún no existe (se crea al inicio de la ejecución)
        if G is None:
            while True:
                try:
                    # Solicita al usuario la cantidad de nodos para la red
                    n = int(input(f"¿Cuántos nodos quieres en la red? (mínimo 3, máximo {num_ciudades}): "))
                    # Verifica si la cantidad de nodos está dentro del rango permitido
                    if n >= 3 and n <= num_ciudades:
                        G = crear_red_dispositivos(n) # Crea la red de dispositivos
                        break # Sale del bucle si la cantidad de nodos es válida
                    else:
                        print(f"Cantidad de nodos no disponible. Por favor, ingresa un número entre 3 y {num_ciudades}.")
                except ValueError:
                    # Maneja el error si la entrada no es un número entero
                    print("Entrada inválida. Por favor, ingresa un número entero.")

        print("\n--- Menú Principal ---")
        print("1. Enviar un mensaje")
        print("3. Salir")
        # Solicita al usuario que seleccione una opción del menú
        opcion_menu_principal = input("Selecciona una opción: ")

        # Procesa la opción seleccionada por el usuario
        if opcion_menu_principal == '1':
            configurar_y_enviar_mensaje_unico(G) # Llama a la función para configurar y enviar un mensaje único
        elif opcion_menu_principal == '3':
            print("Saliendo del simulador. ¡Hasta luego!") # Mensaje de despedida
            break # Sale del bucle principal para terminar el programa
        else:
            print("Opción no válida. Por favor, selecciona 1 o 3.") # Mensaje para opción inválida


if __name__ == "__main__":
    main() # Ejecuta la función principal si el script se ejecuta directamente


=== Simulador de Red Cifrada con RSA + Vigenère ===
¿Cuántos nodos quieres en la red? (mínimo 3, máximo 27): 16

--- Menú Principal ---
1. Enviar un mensaje
3. Salir
Selecciona una opción: 1

Nodos disponibles: [(0, 'Neiva'), (1, 'Bogotá'), (2, 'Quibdó'), (3, 'Bucaramanga'), (4, 'San Andrés'), (5, 'Tunja'), (6, 'Florencia'), (7, 'Sincelejo'), (8, 'Armenia'), (9, 'Manizales'), (10, 'Leticia'), (11, 'Cali'), (12, 'Riohacha'), (13, 'Barranquilla'), (14, 'Cúcuta'), (15, 'Cartagena')]

--- Configuración del Mensaje ---
Nodo origen: 5
Nodo destino: 8
Mensaje a enviar: gato
¿Activar espía (MITM)? (s/n): s

📤 Enviando mensaje de nodo 5 a nodo 8:
 - Camino: ['5 (Tunja)', '8 (Armenia)']
 - Clave cifrada (hex): 04a5dc70a771e19ffaee4db4110ea5739c3d1d39820139ec0651931a452cae570380d25aa0b4a36fae5e9f8e2b371947a896915c40f277c5b3b493d2f0584648152eab8729bcf000df2d48c1b7aa9992bec71936c48612b40f4ddaec789146782ae6ad12e3a043d4ca3b6bc15841e743837e392abc16eba941eca79275e9313978408cc263627ea30b5c62ef0573035b3


--- Menú Principal ---
1. Enviar un mensaje
3. Salir
Selecciona una opción: 3
Saliendo del simulador. ¡Hasta luego!
